In [1]:
import utils.sql as sql_tools
import pandas as pd

In [2]:
cnx, cursor = sql_tools.create_session_cursor()

query = '''
SELECT od.order_id, od.food_id, food_name, food_type, quantity, price, order_time from 
order_details od inner join food inner join `order`
ON od.food_id = food.food_id AND od.order_id = `order`.order_id
WHERE od.order_id in (
	select order_id from `order` where restaurant_id = 1
);
'''

result = sql_tools.execute_fetchall(cursor, query, tuple())
result = pd.DataFrame(result)

result.columns = ['ORDER_ID', 'FOOD_ID', 'FOOD_NAME', 'FOOD_TYPE', 'QUANTITY', 'PRICE', 'ORDER_TIME']
result = result.astype({'QUANTITY': int})
len(result)
result.head()
# result.dtypes

,ORDER_ID,FOOD_ID,FOOD_NAME,FOOD_TYPE,QUANTITY,PRICE,ORDER_TIME
0,1,1,辣椒炒肉,荤菜,1,42.0,2023-03-04 11:49:05
1,1,2,小炒黄牛肉,荤菜,1,68.0,2023-03-04 11:49:05
2,1,3,剁椒鱼头,荤菜,1,96.0,2023-03-04 11:49:05
3,1,5,烟笋炒腊肉,荤菜,1,48.0,2023-03-04 11:49:05
4,1,6,有机花菜,素菜,1,36.0,2023-03-04 11:49:05


In [3]:
def _process_overturn_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order, then:
        1. Calculate the price * quantity
        2. Group by FOOD_TYPE
        3. Group by date (with viewmode)
    '''
    # step 1
    df['Money'] = df['QUANTITY'] * df['PRICE']
    # step 2
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_TYPE')
    for food_type, sub_df in df1:
        # step 3
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_type] = date_grouped['Money']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    return ans
_process_overturn_analysis(result, 'D')

,汤,素菜,荤菜
2023-03-01,2840.0,2280.0,7366.0
2023-03-02,3012.0,2324.0,6996.0
2023-03-03,3108.0,2790.0,8242.0
2023-03-04,3004.0,2538.0,6458.0
2023-03-05,2792.0,2624.0,6302.0
2023-03-06,3224.0,2364.0,7014.0
2023-03-07,2256.0,2126.0,5358.0
2023-03-08,3608.0,2576.0,7628.0
2023-03-09,3220.0,2628.0,7256.0
2023-03-10,2788.0,2208.0,6072.0


In [4]:
def _process_overturn_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order, then:
        1. Calculate the price * quantity
        2. Group by date (with viewmode)
    '''
    # step 1
    df['Money'] = df['QUANTITY'] * df['PRICE']
    # step 2
    ans = df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode))['Money'].sum(numeric_only=True)
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    return ans
_process_overturn_analysis(result, 'D')

2023-03-01    12486.0
2023-03-02    12332.0
2023-03-03    14140.0
2023-03-04    12000.0
2023-03-05    11718.0
2023-03-06    12602.0
2023-03-07     9740.0
2023-03-08    13812.0
2023-03-09    13104.0
2023-03-10    11068.0
2023-03-11    11802.0
2023-03-12    13306.0
2023-03-13    11798.0
2023-03-14    12888.0
2023-03-15    15258.0
Name: Money, dtype: float64

In [5]:
import datetime
def _get_range_turnover(df, start_date, end_date):
    df['Money'] = df['QUANTITY'] * df['PRICE']
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    return df.loc[mask]['Money'].sum()
_get_range_turnover(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

87778.0

In [6]:
def _get_range_orders(df, start_date, end_date):
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    df = df.drop_duplicates(subset=['ORDER_ID'])
    return df.loc[mask]['ORDER_ID'].count()
_get_range_orders(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

279

In [7]:
def _get_range_dishes(df, start_date, end_date):
    mask = (df['ORDER_TIME'] > start_date) & (df['ORDER_TIME'] < end_date)
    return df.loc[mask]['ORDER_ID'].count()
_get_range_dishes(result, datetime.datetime(year=2023, month=3, day=15) - datetime.timedelta(days=7),
                    datetime.datetime(year=2023, month=3, day=15))

1671

In [8]:
def _process_dish_analysis(df, viewmode='D'):
    ''' Processing for the overturn analysis. This function receives food order,
        Then group by food name and date respectively.
    '''
    ans = pd.DataFrame()
    df1 = df.groupby('FOOD_NAME')
    for food_name, sub_df in df1:
        date_grouped = sub_df.groupby(pd.Grouper(key='ORDER_TIME', axis=0, freq=viewmode)).sum(numeric_only=True)
        ans[food_name] = date_grouped['QUANTITY']
    ans.index = [ans.index[i].to_pydatetime().strftime('%Y-%m-%d') for i in range(len(ans.index))]
    print(list(ans.columns[:2]))
    return ans
_process_dish_analysis(result)[['剁椒鱼头', '小炒黄牛肉']]

['剁椒鱼头', '口味鸡']


,剁椒鱼头,小炒黄牛肉
2023-03-01,22,19
2023-03-02,18,22
2023-03-03,26,29
2023-03-04,17,20
2023-03-05,13,19
2023-03-06,17,23
2023-03-07,13,20
2023-03-08,22,28
2023-03-09,18,27
2023-03-10,14,23


In [43]:
def _get_best_selling_dish(df, best_n=3):
    ''' Get the `best_n` selling dishes. '''
    best_n = min(best_n, 3)
    retval = df.groupby('FOOD_ID').count().sort_values(by=['ORDER_ID'], ascending=False).head(best_n)
    idx, value = retval.index, retval['ORDER_ID']
    retval = result.iloc[idx][['FOOD_NAME', 'FOOD_TYPE']]
    retval['SELLING'] = value   
    return retval
 
_get_best_selling_dish(result)  

,FOOD_NAME,FOOD_TYPE,SELLING
10,剁椒鱼头,荤菜,503
1,小炒黄牛肉,荤菜,485
6,茶树菇土鸡汤,汤,476
